# MLP

In [ ]:
pip install pandas scikit-learn openml torch

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.6 MB/s eta 0:00:00
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=94c69f7ae318707bde9b7b73ce8971ec00fb361063e7c1276aa082efb1b70d57
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built liac-arff


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import openml
import pandas as pd

# Function for random parameter generation
def get_random_parameters(seed):
    rs = np.random.RandomState(seed)
    params = {
        "hidden_dim": rs.randint(10, 101),
        "n_layers": rs.randint(2, 6),
        "learning_rate": rs.uniform(0.00005, 0.0005),  # Adjusted lower learning rate range
    }
    return params

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data from OpenML using task ID
def load_openml_data(task_id):

    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)

    # Convert categorical features to numeric using one-hot encoding
    X = pd.get_dummies(X, drop_first=True)

    # Fill NaN values with column means and Inf with large finite values
    X = X.fillna(X.mean())
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    X = X.dropna()  # Optionally drop remaining NaNs if any

    # Convert to numpy
    X = X.to_numpy().astype(np.float32)
    y = pd.factorize(y)[0]

    return X, y


# Define MLP model
def initialize_mlp(n_layers, input_dim, hidden_dim, output_dim, objective):
    layers = [nn.Linear(input_dim, hidden_dim), nn.ReLU()]
    layers.extend([nn.Linear(hidden_dim, hidden_dim), nn.ReLU()] * (n_layers - 1))
    layers.append(nn.Linear(hidden_dim, output_dim))
    model = nn.Sequential(*layers)
    return model

# Training function with gradient clipping
def train_mlp(model, X_train, y_train, X_val, y_val, args, params):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = optim.AdamW(model.parameters(), lr=params["learning_rate"])
    loss_func = nn.CrossEntropyLoss()

    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    train_loader = DataLoader(dataset=train_dataset, batch_size=args["batch_size"], shuffle=True)

    val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
    val_loader = DataLoader(dataset=val_dataset, batch_size=args["val_batch_size"], shuffle=False)

    for epoch in range(args["epochs"]):
        model.train()
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            output = model(batch_X)
            loss = loss_func(output, batch_y)
            loss.backward()

            # Apply gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                output = model(batch_X)
                val_loss += loss_func(output, batch_y).item()

        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")

    return model

# Prediction function
def predict_mlp(model, X, args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    with torch.no_grad():
        output = model(X_tensor)
        predictions = output.argmax(dim=1).cpu().numpy()

    return predictions

# Set hyperparameters and arguments
args = {
    "objective": "classification",
    "batch_size": 64,
    "val_batch_size": 64,
    "epochs": 10,
    "num_features": None,
    "num_classes": None,
}

seed = 42
params = get_random_parameters(seed)



Using device: cpu


# Task ID1: 14965

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=14965)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.2617
Epoch 2, Validation Loss: 0.2339
Epoch 3, Validation Loss: 0.2280
Epoch 4, Validation Loss: 0.2242
Epoch 5, Validation Loss: 0.2222
Epoch 6, Validation Loss: 0.2206
Epoch 7, Validation Loss: 0.2194
Epoch 8, Validation Loss: 0.2180
Epoch 9, Validation Loss: 0.2175
Epoch 10, Validation Loss: 0.2166
Validation Accuracy: 0.9035


# Task ID2: 9977

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9977)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.1591
Epoch 2, Validation Loss: 0.1377
Epoch 3, Validation Loss: 0.1296
Epoch 4, Validation Loss: 0.1253
Epoch 5, Validation Loss: 0.1221
Epoch 6, Validation Loss: 0.1208
Epoch 7, Validation Loss: 0.1183
Epoch 8, Validation Loss: 0.1173
Epoch 9, Validation Loss: 0.1144
Epoch 10, Validation Loss: 0.1163
Validation Accuracy: 0.9542


# Task ID3: 34539

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=34539)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.2228
Epoch 2, Validation Loss: 0.1846
Epoch 3, Validation Loss: 0.1945
Epoch 4, Validation Loss: 0.2289
Epoch 5, Validation Loss: 0.2576
Epoch 6, Validation Loss: 0.2801
Epoch 7, Validation Loss: 0.3096
Epoch 8, Validation Loss: 0.3476
Epoch 9, Validation Loss: 0.3596
Epoch 10, Validation Loss: 0.4040
Validation Accuracy: 0.9362


# Task ID4: 146606

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=146606)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6401
Epoch 2, Validation Loss: 0.6242
Epoch 3, Validation Loss: 0.6142
Epoch 4, Validation Loss: 0.6077
Epoch 5, Validation Loss: 0.6024
Epoch 6, Validation Loss: 0.5982
Epoch 7, Validation Loss: 0.5935
Epoch 8, Validation Loss: 0.5907
Epoch 9, Validation Loss: 0.5875
Epoch 10, Validation Loss: 0.5851
Validation Accuracy: 0.6917


# Task ID5: 7592



In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=7592)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.3434
Epoch 2, Validation Loss: 0.3242
Epoch 3, Validation Loss: 0.3154
Epoch 4, Validation Loss: 0.3106
Epoch 5, Validation Loss: 0.3086
Epoch 6, Validation Loss: 0.3070
Epoch 7, Validation Loss: 0.3053
Epoch 8, Validation Loss: 0.3050
Epoch 9, Validation Loss: 0.3043
Epoch 10, Validation Loss: 0.3035
Validation Accuracy: 0.8590


# Task ID6: 146195



In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=146195)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6165
Epoch 2, Validation Loss: 0.5790
Epoch 3, Validation Loss: 0.5577
Epoch 4, Validation Loss: 0.5400
Epoch 5, Validation Loss: 0.5217
Epoch 6, Validation Loss: 0.5079
Epoch 7, Validation Loss: 0.4953
Epoch 8, Validation Loss: 0.4868
Epoch 9, Validation Loss: 0.4793
Epoch 10, Validation Loss: 0.4734
Validation Accuracy: 0.8105


# Task ID7: 167119



In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=167119)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.7152
Epoch 2, Validation Loss: 0.6600
Epoch 3, Validation Loss: 0.6155
Epoch 4, Validation Loss: 0.5792
Epoch 5, Validation Loss: 0.5527
Epoch 6, Validation Loss: 0.5334
Epoch 7, Validation Loss: 0.5176
Epoch 8, Validation Loss: 0.5044
Epoch 9, Validation Loss: 0.4946
Epoch 10, Validation Loss: 0.4844
Validation Accuracy: 0.8017


# Task ID8: 167120



In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=167120)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6932
Epoch 2, Validation Loss: 0.6927
Epoch 3, Validation Loss: 0.6927
Epoch 4, Validation Loss: 0.6926
Epoch 5, Validation Loss: 0.6926
Epoch 6, Validation Loss: 0.6927
Epoch 7, Validation Loss: 0.6926
Epoch 8, Validation Loss: 0.6925
Epoch 9, Validation Loss: 0.6927
Epoch 10, Validation Loss: 0.6928
Validation Accuracy: 0.5166


# Task ID11: 	168331



In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=168331)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.3411
Epoch 2, Validation Loss: 1.2322
Epoch 3, Validation Loss: 1.1965
Epoch 4, Validation Loss: 1.1750
Epoch 5, Validation Loss: 1.1582
Epoch 6, Validation Loss: 1.1426
Epoch 7, Validation Loss: 1.1332
Epoch 8, Validation Loss: 1.1247
Epoch 9, Validation Loss: 1.1153
Epoch 10, Validation Loss: 1.1086
Validation Accuracy: 0.6020


# Task ID12: 	168330



In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=168330)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.8471
Epoch 2, Validation Loss: 0.8239
Epoch 3, Validation Loss: 0.8099
Epoch 4, Validation Loss: 0.7987
Epoch 5, Validation Loss: 0.7902
Epoch 6, Validation Loss: 0.7818
Epoch 7, Validation Loss: 0.7741
Epoch 8, Validation Loss: 0.7683
Epoch 9, Validation Loss: 0.7628
Epoch 10, Validation Loss: 0.7589
Validation Accuracy: 0.6781


# Task ID13: 	168335



In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=168335)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.2862
Epoch 2, Validation Loss: 0.2583
Epoch 3, Validation Loss: 0.2466
Epoch 4, Validation Loss: 0.2383
Epoch 5, Validation Loss: 0.2342
Epoch 6, Validation Loss: 0.2281
Epoch 7, Validation Loss: 0.2243
Epoch 8, Validation Loss: 0.2217
Epoch 9, Validation Loss: 0.2180
Epoch 10, Validation Loss: 0.2138
Validation Accuracy: 0.9114



# Task ID16: 	146212



In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=146212)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.2034
Epoch 2, Validation Loss: 0.1067
Epoch 3, Validation Loss: 0.0710
Epoch 4, Validation Loss: 0.0523
Epoch 5, Validation Loss: 0.0421
Epoch 6, Validation Loss: 0.0360
Epoch 7, Validation Loss: 0.0320
Epoch 8, Validation Loss: 0.0293
Epoch 9, Validation Loss: 0.0272
Epoch 10, Validation Loss: 0.0261
Validation Accuracy: 0.9962


# Task ID19: 	168868



In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=168868)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.0426
Epoch 2, Validation Loss: 0.0381
Epoch 3, Validation Loss: 0.0379
Epoch 4, Validation Loss: 0.0369
Epoch 5, Validation Loss: 0.0378
Epoch 6, Validation Loss: 0.0383
Epoch 7, Validation Loss: 0.0405
Epoch 8, Validation Loss: 0.0446
Epoch 9, Validation Loss: 0.0424
Epoch 10, Validation Loss: 0.0426
Validation Accuracy: 0.9909


# Task ID20: 31

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=31)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6796
Epoch 2, Validation Loss: 0.6699
Epoch 3, Validation Loss: 0.6615
Epoch 4, Validation Loss: 0.6542
Epoch 5, Validation Loss: 0.6480
Epoch 6, Validation Loss: 0.6422
Epoch 7, Validation Loss: 0.6373
Epoch 8, Validation Loss: 0.6326
Epoch 9, Validation Loss: 0.6285
Epoch 10, Validation Loss: 0.6248
Validation Accuracy: 0.7050


# Task ID21: 10101

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=10101)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6583
Epoch 2, Validation Loss: 0.6488
Epoch 3, Validation Loss: 0.6396
Epoch 4, Validation Loss: 0.6304
Epoch 5, Validation Loss: 0.6215
Epoch 6, Validation Loss: 0.6122
Epoch 7, Validation Loss: 0.6034
Epoch 8, Validation Loss: 0.5944
Epoch 9, Validation Loss: 0.5860
Epoch 10, Validation Loss: 0.5780
Validation Accuracy: 0.7533


# Task ID22: 	3913

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=3913)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6336
Epoch 2, Validation Loss: 0.6247
Epoch 3, Validation Loss: 0.6157
Epoch 4, Validation Loss: 0.6071
Epoch 5, Validation Loss: 0.5979
Epoch 6, Validation Loss: 0.5885
Epoch 7, Validation Loss: 0.5789
Epoch 8, Validation Loss: 0.5693
Epoch 9, Validation Loss: 0.5597
Epoch 10, Validation Loss: 0.5496
Validation Accuracy: 0.8095


# Task ID23: 3

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=3)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6705
Epoch 2, Validation Loss: 0.6494
Epoch 3, Validation Loss: 0.6219
Epoch 4, Validation Loss: 0.5850
Epoch 5, Validation Loss: 0.5380
Epoch 6, Validation Loss: 0.4850
Epoch 7, Validation Loss: 0.4317
Epoch 8, Validation Loss: 0.3815
Epoch 9, Validation Loss: 0.3370
Epoch 10, Validation Loss: 0.2992
Validation Accuracy: 0.9062


# Task ID24: 3917

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=3917)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6958
Epoch 2, Validation Loss: 0.6509
Epoch 3, Validation Loss: 0.6055
Epoch 4, Validation Loss: 0.5575
Epoch 5, Validation Loss: 0.5046
Epoch 6, Validation Loss: 0.4568
Epoch 7, Validation Loss: 0.4231
Epoch 8, Validation Loss: 0.4041
Epoch 9, Validation Loss: 0.3947
Epoch 10, Validation Loss: 0.3909
Validation Accuracy: 0.8412


# Task ID25: 9957

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9957)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6828
Epoch 2, Validation Loss: 0.6638
Epoch 3, Validation Loss: 0.6472
Epoch 4, Validation Loss: 0.6314
Epoch 5, Validation Loss: 0.6163
Epoch 6, Validation Loss: 0.6010
Epoch 7, Validation Loss: 0.5852
Epoch 8, Validation Loss: 0.5687
Epoch 9, Validation Loss: 0.5516
Epoch 10, Validation Loss: 0.5351
Validation Accuracy: 0.7630


# Task ID26: 9946

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9946)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6439
Epoch 2, Validation Loss: 0.6249
Epoch 3, Validation Loss: 0.6068
Epoch 4, Validation Loss: 0.5884
Epoch 5, Validation Loss: 0.5695
Epoch 6, Validation Loss: 0.5504
Epoch 7, Validation Loss: 0.5311
Epoch 8, Validation Loss: 0.5119
Epoch 9, Validation Loss: 0.4920
Epoch 10, Validation Loss: 0.4710
Validation Accuracy: 0.9474


# Task ID27: 3918

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=3918)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6102
Epoch 2, Validation Loss: 0.5890
Epoch 3, Validation Loss: 0.5686
Epoch 4, Validation Loss: 0.5492
Epoch 5, Validation Loss: 0.5296
Epoch 6, Validation Loss: 0.5104
Epoch 7, Validation Loss: 0.4907
Epoch 8, Validation Loss: 0.4704
Epoch 9, Validation Loss: 0.4500
Epoch 10, Validation Loss: 0.4292
Validation Accuracy: 0.9144


# Task ID28: 3903

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=3903)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6419
Epoch 2, Validation Loss: 0.6060
Epoch 3, Validation Loss: 0.5697
Epoch 4, Validation Loss: 0.5309
Epoch 5, Validation Loss: 0.4889
Epoch 6, Validation Loss: 0.4463
Epoch 7, Validation Loss: 0.4095
Epoch 8, Validation Loss: 0.3783
Epoch 9, Validation Loss: 0.3557
Epoch 10, Validation Loss: 0.3417
Validation Accuracy: 0.8978


# Task ID29: 37

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=37)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6940
Epoch 2, Validation Loss: 0.6859
Epoch 3, Validation Loss: 0.6777
Epoch 4, Validation Loss: 0.6699
Epoch 5, Validation Loss: 0.6616
Epoch 6, Validation Loss: 0.6538
Epoch 7, Validation Loss: 0.6454
Epoch 8, Validation Loss: 0.6369
Epoch 9, Validation Loss: 0.6284
Epoch 10, Validation Loss: 0.6191
Validation Accuracy: 0.7143


# Task ID30: 9971

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9971)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6758
Epoch 2, Validation Loss: 0.6643
Epoch 3, Validation Loss: 0.6537
Epoch 4, Validation Loss: 0.6440
Epoch 5, Validation Loss: 0.6347
Epoch 6, Validation Loss: 0.6262
Epoch 7, Validation Loss: 0.6183
Epoch 8, Validation Loss: 0.6105
Epoch 9, Validation Loss: 0.6031
Epoch 10, Validation Loss: 0.5962
Validation Accuracy: 0.7436


# Task ID31: 9952

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9952)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.5894
Epoch 2, Validation Loss: 0.5326
Epoch 3, Validation Loss: 0.4930
Epoch 4, Validation Loss: 0.4664
Epoch 5, Validation Loss: 0.4503
Epoch 6, Validation Loss: 0.4404
Epoch 7, Validation Loss: 0.4341
Epoch 8, Validation Loss: 0.4290
Epoch 9, Validation Loss: 0.4250
Epoch 10, Validation Loss: 0.4212
Validation Accuracy: 0.7928


# Task ID32: 3902

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=3902)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6374
Epoch 2, Validation Loss: 0.6065
Epoch 3, Validation Loss: 0.5770
Epoch 4, Validation Loss: 0.5469
Epoch 5, Validation Loss: 0.5155
Epoch 6, Validation Loss: 0.4833
Epoch 7, Validation Loss: 0.4533
Epoch 8, Validation Loss: 0.4270
Epoch 9, Validation Loss: 0.4056
Epoch 10, Validation Loss: 0.3884
Validation Accuracy: 0.8562


# Task ID33: 49

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=49)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6989
Epoch 2, Validation Loss: 0.6880
Epoch 3, Validation Loss: 0.6777
Epoch 4, Validation Loss: 0.6678
Epoch 5, Validation Loss: 0.6581
Epoch 6, Validation Loss: 0.6489
Epoch 7, Validation Loss: 0.6397
Epoch 8, Validation Loss: 0.6311
Epoch 9, Validation Loss: 0.6227
Epoch 10, Validation Loss: 0.6157
Validation Accuracy: 0.6510


# Task ID34: 43

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=43)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6400
Epoch 2, Validation Loss: 0.5789
Epoch 3, Validation Loss: 0.4994
Epoch 4, Validation Loss: 0.4110
Epoch 5, Validation Loss: 0.3360
Epoch 6, Validation Loss: 0.2877
Epoch 7, Validation Loss: 0.2573
Epoch 8, Validation Loss: 0.2394
Epoch 9, Validation Loss: 0.2264
Epoch 10, Validation Loss: 0.2177
Validation Accuracy: 0.9327


# Task ID35: 9978

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9978)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.5707
Epoch 2, Validation Loss: 0.4667
Epoch 3, Validation Loss: 0.3744
Epoch 4, Validation Loss: 0.3052
Epoch 5, Validation Loss: 0.2637
Epoch 6, Validation Loss: 0.2391
Epoch 7, Validation Loss: 0.2235
Epoch 8, Validation Loss: 0.2130
Epoch 9, Validation Loss: 0.2048
Epoch 10, Validation Loss: 0.1986
Validation Accuracy: 0.9349


# Task ID36: 10093

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=10093)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6556
Epoch 2, Validation Loss: 0.6368
Epoch 3, Validation Loss: 0.6176
Epoch 4, Validation Loss: 0.5975
Epoch 5, Validation Loss: 0.5765
Epoch 6, Validation Loss: 0.5538
Epoch 7, Validation Loss: 0.5295
Epoch 8, Validation Loss: 0.5030
Epoch 9, Validation Loss: 0.4755
Epoch 10, Validation Loss: 0.4461
Validation Accuracy: 0.8618


# Task ID37: 219

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=219)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.5211
Epoch 2, Validation Loss: 0.4844
Epoch 3, Validation Loss: 0.4739
Epoch 4, Validation Loss: 0.4685
Epoch 5, Validation Loss: 0.4635
Epoch 6, Validation Loss: 0.4606
Epoch 7, Validation Loss: 0.4582
Epoch 8, Validation Loss: 0.4547
Epoch 9, Validation Loss: 0.4522
Epoch 10, Validation Loss: 0.4505
Validation Accuracy: 0.7888


# Task ID38: 9976

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9976)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6944
Epoch 2, Validation Loss: 0.6916
Epoch 3, Validation Loss: 0.6885
Epoch 4, Validation Loss: 0.6854
Epoch 5, Validation Loss: 0.6819
Epoch 6, Validation Loss: 0.6773
Epoch 7, Validation Loss: 0.6732
Epoch 8, Validation Loss: 0.6692
Epoch 9, Validation Loss: 0.6670
Epoch 10, Validation Loss: 0.6687
Validation Accuracy: 0.5423


# Task ID39: 6

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=6)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 3.0598
Epoch 2, Validation Loss: 2.5152
Epoch 3, Validation Loss: 1.9552
Epoch 4, Validation Loss: 1.6047
Epoch 5, Validation Loss: 1.3912
Epoch 6, Validation Loss: 1.2533
Epoch 7, Validation Loss: 1.1598
Epoch 8, Validation Loss: 1.0897
Epoch 9, Validation Loss: 1.0362
Epoch 10, Validation Loss: 0.9918
Validation Accuracy: 0.7130


# Task ID40: 53

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=53)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.3758
Epoch 2, Validation Loss: 1.3669
Epoch 3, Validation Loss: 1.3582
Epoch 4, Validation Loss: 1.3493
Epoch 5, Validation Loss: 1.3408
Epoch 6, Validation Loss: 1.3316
Epoch 7, Validation Loss: 1.3224
Epoch 8, Validation Loss: 1.3135
Epoch 9, Validation Loss: 1.3037
Epoch 10, Validation Loss: 1.2932
Validation Accuracy: 0.4471


# Task ID41: 11

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=11)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.0777
Epoch 2, Validation Loss: 1.0609
Epoch 3, Validation Loss: 1.0445
Epoch 4, Validation Loss: 1.0284
Epoch 5, Validation Loss: 1.0130
Epoch 6, Validation Loss: 0.9978
Epoch 7, Validation Loss: 0.9828
Epoch 8, Validation Loss: 0.9681
Epoch 9, Validation Loss: 0.9534
Epoch 10, Validation Loss: 0.9387
Validation Accuracy: 0.5760


# Task ID42: 15

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=15)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6640
Epoch 2, Validation Loss: 0.6463
Epoch 3, Validation Loss: 0.6286
Epoch 4, Validation Loss: 0.6108
Epoch 5, Validation Loss: 0.5924
Epoch 6, Validation Loss: 0.5734
Epoch 7, Validation Loss: 0.5538
Epoch 8, Validation Loss: 0.5335
Epoch 9, Validation Loss: 0.5123
Epoch 10, Validation Loss: 0.4902
Validation Accuracy: 0.9643


# Task ID43: 16

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=16)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.2873
Epoch 2, Validation Loss: 2.2541
Epoch 3, Validation Loss: 2.2198
Epoch 4, Validation Loss: 2.1813
Epoch 5, Validation Loss: 2.1356
Epoch 6, Validation Loss: 2.0804
Epoch 7, Validation Loss: 2.0135
Epoch 8, Validation Loss: 1.9330
Epoch 9, Validation Loss: 1.8391
Epoch 10, Validation Loss: 1.7345
Validation Accuracy: 0.7475


# Task ID44: 14

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=14)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.2667
Epoch 2, Validation Loss: 2.2367
Epoch 3, Validation Loss: 2.2022
Epoch 4, Validation Loss: 2.1601
Epoch 5, Validation Loss: 2.1078
Epoch 6, Validation Loss: 2.0448
Epoch 7, Validation Loss: 1.9664
Epoch 8, Validation Loss: 1.8777
Epoch 9, Validation Loss: 1.7794
Epoch 10, Validation Loss: 1.6748
Validation Accuracy: 0.6650


# Task ID45: 32

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=32)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.0543
Epoch 2, Validation Loss: 1.5984
Epoch 3, Validation Loss: 1.0985
Epoch 4, Validation Loss: 0.7730
Epoch 5, Validation Loss: 0.5953
Epoch 6, Validation Loss: 0.4847
Epoch 7, Validation Loss: 0.4060
Epoch 8, Validation Loss: 0.3460
Epoch 9, Validation Loss: 0.2996
Epoch 10, Validation Loss: 0.2613
Validation Accuracy: 0.9395


# Task ID46: 3549

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=3549)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.3434
Epoch 2, Validation Loss: 1.3177
Epoch 3, Validation Loss: 1.2914
Epoch 4, Validation Loss: 1.2630
Epoch 5, Validation Loss: 1.2317
Epoch 6, Validation Loss: 1.1964
Epoch 7, Validation Loss: 1.1564
Epoch 8, Validation Loss: 1.1114
Epoch 9, Validation Loss: 1.0612
Epoch 10, Validation Loss: 1.0068
Validation Accuracy: 0.8698


# Task ID47: 12

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=12)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.2062
Epoch 2, Validation Loss: 2.0821
Epoch 3, Validation Loss: 1.9274
Epoch 4, Validation Loss: 1.7305
Epoch 5, Validation Loss: 1.5005
Epoch 6, Validation Loss: 1.2659
Epoch 7, Validation Loss: 1.0485
Epoch 8, Validation Loss: 0.8616
Epoch 9, Validation Loss: 0.7098
Epoch 10, Validation Loss: 0.5956
Validation Accuracy: 0.9050


# Task ID48: 9981

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9981)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.1943
Epoch 2, Validation Loss: 2.1784
Epoch 3, Validation Loss: 2.1623
Epoch 4, Validation Loss: 2.1441
Epoch 5, Validation Loss: 2.1234
Epoch 6, Validation Loss: 2.0991
Epoch 7, Validation Loss: 2.0694
Epoch 8, Validation Loss: 2.0333
Epoch 9, Validation Loss: 1.9903
Epoch 10, Validation Loss: 1.9388
Validation Accuracy: 0.6343


# Task ID49: 18

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=18)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.2746
Epoch 2, Validation Loss: 2.2305
Epoch 3, Validation Loss: 2.1823
Epoch 4, Validation Loss: 2.1290
Epoch 5, Validation Loss: 2.0712
Epoch 6, Validation Loss: 2.0048
Epoch 7, Validation Loss: 1.9350
Epoch 8, Validation Loss: 1.8620
Epoch 9, Validation Loss: 1.7887
Epoch 10, Validation Loss: 1.7161
Validation Accuracy: 0.4850


# Task ID50: 28

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=28)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.1857
Epoch 2, Validation Loss: 2.0054
Epoch 3, Validation Loss: 1.7261
Epoch 4, Validation Loss: 1.3772
Epoch 5, Validation Loss: 1.0364
Epoch 6, Validation Loss: 0.7710
Epoch 7, Validation Loss: 0.5834
Epoch 8, Validation Loss: 0.4555
Epoch 9, Validation Loss: 0.3731
Epoch 10, Validation Loss: 0.3176
Validation Accuracy: 0.9262


# Task ID51: 2074

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=2074)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.4955
Epoch 2, Validation Loss: 1.1203
Epoch 3, Validation Loss: 0.8331
Epoch 4, Validation Loss: 0.6529
Epoch 5, Validation Loss: 0.5382
Epoch 6, Validation Loss: 0.4693
Epoch 7, Validation Loss: 0.4304
Epoch 8, Validation Loss: 0.4069
Epoch 9, Validation Loss: 0.3908
Epoch 10, Validation Loss: 0.3777
Validation Accuracy: 0.8429


# Task ID52: 29

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=29)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.7111
Epoch 2, Validation Loss: 0.7075
Epoch 3, Validation Loss: 0.7037
Epoch 4, Validation Loss: 0.7000
Epoch 5, Validation Loss: 0.6961
Epoch 6, Validation Loss: 0.6926
Epoch 7, Validation Loss: 0.6879
Epoch 8, Validation Loss: 0.6830
Epoch 9, Validation Loss: 0.6778
Epoch 10, Validation Loss: 0.6720
Validation Accuracy: 0.5290


# Task ID53: 45

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=45)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.0605
Epoch 2, Validation Loss: 1.0213
Epoch 3, Validation Loss: 0.9760
Epoch 4, Validation Loss: 0.9206
Epoch 5, Validation Loss: 0.8491
Epoch 6, Validation Loss: 0.7591
Epoch 7, Validation Loss: 0.6581
Epoch 8, Validation Loss: 0.5572
Epoch 9, Validation Loss: 0.4657
Epoch 10, Validation Loss: 0.3918
Validation Accuracy: 0.8997


# Task ID54: 125922

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=125922)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.2950
Epoch 2, Validation Loss: 2.1239
Epoch 3, Validation Loss: 1.8690
Epoch 4, Validation Loss: 1.6383
Epoch 5, Validation Loss: 1.4472
Epoch 6, Validation Loss: 1.2707
Epoch 7, Validation Loss: 1.1165
Epoch 8, Validation Loss: 0.9866
Epoch 9, Validation Loss: 0.8817
Epoch 10, Validation Loss: 0.7872
Validation Accuracy: 0.7736


# Task ID55: 9960

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9960)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.3750
Epoch 2, Validation Loss: 1.2335
Epoch 3, Validation Loss: 1.1062
Epoch 4, Validation Loss: 1.0164
Epoch 5, Validation Loss: 0.9501
Epoch 6, Validation Loss: 0.8935
Epoch 7, Validation Loss: 0.8453
Epoch 8, Validation Loss: 0.8035
Epoch 9, Validation Loss: 0.7678
Epoch 10, Validation Loss: 0.7366
Validation Accuracy: 0.7033


# Task ID56: 9964

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9964)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.2544
Epoch 2, Validation Loss: 2.2103
Epoch 3, Validation Loss: 2.1569
Epoch 4, Validation Loss: 2.0902
Epoch 5, Validation Loss: 2.0091
Epoch 6, Validation Loss: 1.9161
Epoch 7, Validation Loss: 1.8119
Epoch 8, Validation Loss: 1.6974
Epoch 9, Validation Loss: 1.5791
Epoch 10, Validation Loss: 1.4525
Validation Accuracy: 0.6991


# Task ID57: 22

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=22)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.2549
Epoch 2, Validation Loss: 2.2111
Epoch 3, Validation Loss: 2.1641
Epoch 4, Validation Loss: 2.1103
Epoch 5, Validation Loss: 2.0456
Epoch 6, Validation Loss: 1.9719
Epoch 7, Validation Loss: 1.8883
Epoch 8, Validation Loss: 1.7946
Epoch 9, Validation Loss: 1.6943
Epoch 10, Validation Loss: 1.5930
Validation Accuracy: 0.6400


# Task ID58: 2079

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=2079)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.6230
Epoch 2, Validation Loss: 1.6110
Epoch 3, Validation Loss: 1.6001
Epoch 4, Validation Loss: 1.5892
Epoch 5, Validation Loss: 1.5793
Epoch 6, Validation Loss: 1.5685
Epoch 7, Validation Loss: 1.5575
Epoch 8, Validation Loss: 1.5465
Epoch 9, Validation Loss: 1.5355
Epoch 10, Validation Loss: 1.5239
Validation Accuracy: 0.4189


# Task ID59: 14969

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=14969)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.4949
Epoch 2, Validation Loss: 1.4222
Epoch 3, Validation Loss: 1.3568
Epoch 4, Validation Loss: 1.3166
Epoch 5, Validation Loss: 1.2917
Epoch 6, Validation Loss: 1.2732
Epoch 7, Validation Loss: 1.2587
Epoch 8, Validation Loss: 1.2465
Epoch 9, Validation Loss: 1.2376
Epoch 10, Validation Loss: 1.2295
Validation Accuracy: 0.4942


# Task ID60: 3560

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=3560)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.7911
Epoch 2, Validation Loss: 1.7899
Epoch 3, Validation Loss: 1.7889
Epoch 4, Validation Loss: 1.7883
Epoch 5, Validation Loss: 1.7875
Epoch 6, Validation Loss: 1.7870
Epoch 7, Validation Loss: 1.7861
Epoch 8, Validation Loss: 1.7857
Epoch 9, Validation Loss: 1.7852
Epoch 10, Validation Loss: 1.7847
Validation Accuracy: 0.2062


# Task ID61: 14952

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=14952)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.5276
Epoch 2, Validation Loss: 0.2956
Epoch 3, Validation Loss: 0.2029
Epoch 4, Validation Loss: 0.1713
Epoch 5, Validation Loss: 0.1561
Epoch 6, Validation Loss: 0.1467
Epoch 7, Validation Loss: 0.1411
Epoch 8, Validation Loss: 0.1361
Epoch 9, Validation Loss: 0.1325
Epoch 10, Validation Loss: 0.1294
Validation Accuracy: 0.9466


# Task ID62: 125920

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=125920)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6992
Epoch 2, Validation Loss: 0.6978
Epoch 3, Validation Loss: 0.6960
Epoch 4, Validation Loss: 0.6944
Epoch 5, Validation Loss: 0.6931
Epoch 6, Validation Loss: 0.6918
Epoch 7, Validation Loss: 0.6906
Epoch 8, Validation Loss: 0.6894
Epoch 9, Validation Loss: 0.6884
Epoch 10, Validation Loss: 0.6875
Validation Accuracy: 0.5500


# Task ID63: 23

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=23)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.0792
Epoch 2, Validation Loss: 1.0700
Epoch 3, Validation Loss: 1.0620
Epoch 4, Validation Loss: 1.0542
Epoch 5, Validation Loss: 1.0474
Epoch 6, Validation Loss: 1.0411
Epoch 7, Validation Loss: 1.0348
Epoch 8, Validation Loss: 1.0299
Epoch 9, Validation Loss: 1.0247
Epoch 10, Validation Loss: 1.0197
Validation Accuracy: 0.4441


# Task ID64: 3904

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=3904)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.5274
Epoch 2, Validation Loss: 0.4472
Epoch 3, Validation Loss: 0.4368
Epoch 4, Validation Loss: 0.4322
Epoch 5, Validation Loss: 0.4301
Epoch 6, Validation Loss: 0.4286
Epoch 7, Validation Loss: 0.4277
Epoch 8, Validation Loss: 0.4278
Epoch 9, Validation Loss: 0.4269
Epoch 10, Validation Loss: 0.4269
Validation Accuracy: 0.8103


# Task ID65: 3022

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=3022)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.4103
Epoch 2, Validation Loss: 2.4044
Epoch 3, Validation Loss: 2.3989
Epoch 4, Validation Loss: 2.3929
Epoch 5, Validation Loss: 2.3870
Epoch 6, Validation Loss: 2.3805
Epoch 7, Validation Loss: 2.3737
Epoch 8, Validation Loss: 2.3668
Epoch 9, Validation Loss: 2.3592
Epoch 10, Validation Loss: 2.3508
Validation Accuracy: 0.2020


# Task ID66: 9985

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9985)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.7212
Epoch 2, Validation Loss: 1.6468
Epoch 3, Validation Loss: 1.5865
Epoch 4, Validation Loss: 1.5565
Epoch 5, Validation Loss: 1.5380
Epoch 6, Validation Loss: 1.5222
Epoch 7, Validation Loss: 1.5085
Epoch 8, Validation Loss: 1.4947
Epoch 9, Validation Loss: 1.4838
Epoch 10, Validation Loss: 1.4735
Validation Accuracy: 0.4379


# Task ID67: 9910

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9910)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6170
Epoch 2, Validation Loss: 0.5535
Epoch 3, Validation Loss: 0.5076
Epoch 4, Validation Loss: 0.4806
Epoch 5, Validation Loss: 0.4689
Epoch 6, Validation Loss: 0.4688
Epoch 7, Validation Loss: 0.4845
Epoch 8, Validation Loss: 0.4881
Epoch 9, Validation Loss: 0.5072
Epoch 10, Validation Loss: 0.5125
Validation Accuracy: 0.7803


# Task ID68: 14970

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=14970)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.7674
Epoch 2, Validation Loss: 0.4253
Epoch 3, Validation Loss: 0.2916
Epoch 4, Validation Loss: 0.2276
Epoch 5, Validation Loss: 0.1757
Epoch 6, Validation Loss: 0.1487
Epoch 7, Validation Loss: 0.1308
Epoch 8, Validation Loss: 0.1237
Epoch 9, Validation Loss: 0.1068
Epoch 10, Validation Loss: 0.0977
Validation Accuracy: 0.9665


# Task ID69: 3021

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=3021)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.5455
Epoch 2, Validation Loss: 0.4186
Epoch 3, Validation Loss: 0.3130
Epoch 4, Validation Loss: 0.2420
Epoch 5, Validation Loss: 0.2021
Epoch 6, Validation Loss: 0.1798
Epoch 7, Validation Loss: 0.1655
Epoch 8, Validation Loss: 0.1548
Epoch 9, Validation Loss: 0.1467
Epoch 10, Validation Loss: 0.1397
Validation Accuracy: 0.9497


# Task ID70: 3481

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=3481)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.6960
Epoch 2, Validation Loss: 1.9094
Epoch 3, Validation Loss: 1.2953
Epoch 4, Validation Loss: 0.8847
Epoch 5, Validation Loss: 0.6628
Epoch 6, Validation Loss: 0.5382
Epoch 7, Validation Loss: 0.4591
Epoch 8, Validation Loss: 0.4028
Epoch 9, Validation Loss: 0.3583
Epoch 10, Validation Loss: 0.3219
Validation Accuracy: 0.9135


# Task ID71: 3573

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=9946)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


# Task ID72: 146824

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=146824)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.1643
Epoch 2, Validation Loss: 2.0326
Epoch 3, Validation Loss: 1.8724
Epoch 4, Validation Loss: 1.6769
Epoch 5, Validation Loss: 1.4650
Epoch 6, Validation Loss: 1.2563
Epoch 7, Validation Loss: 1.0627
Epoch 8, Validation Loss: 0.8854
Epoch 9, Validation Loss: 0.7300
Epoch 10, Validation Loss: 0.6046
Validation Accuracy: 0.9025


# Task ID73: 146820

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=146820)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.4838
Epoch 2, Validation Loss: 0.3455
Epoch 3, Validation Loss: 0.2481
Epoch 4, Validation Loss: 0.2040
Epoch 5, Validation Loss: 0.1864
Epoch 6, Validation Loss: 0.1785
Epoch 7, Validation Loss: 0.1737
Epoch 8, Validation Loss: 0.1702
Epoch 9, Validation Loss: 0.1673
Epoch 10, Validation Loss: 0.1648
Validation Accuracy: 0.9473


# Task ID74: 146822

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=146822)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.9233
Epoch 2, Validation Loss: 1.8733
Epoch 3, Validation Loss: 1.8190
Epoch 4, Validation Loss: 1.7579
Epoch 5, Validation Loss: 1.6870
Epoch 6, Validation Loss: 1.6068
Epoch 7, Validation Loss: 1.5206
Epoch 8, Validation Loss: 1.4356
Epoch 9, Validation Loss: 1.3555
Epoch 10, Validation Loss: 1.2816
Validation Accuracy: 0.6429


# Task ID75: 146195

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=146195)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6205
Epoch 2, Validation Loss: 0.5788
Epoch 3, Validation Loss: 0.5567
Epoch 4, Validation Loss: 0.5365
Epoch 5, Validation Loss: 0.5192
Epoch 6, Validation Loss: 0.5063
Epoch 7, Validation Loss: 0.4956
Epoch 8, Validation Loss: 0.4878
Epoch 9, Validation Loss: 0.4815
Epoch 10, Validation Loss: 0.4753
Validation Accuracy: 0.8122


# Task ID76: 146800

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=146800)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 2.0621
Epoch 2, Validation Loss: 2.0422
Epoch 3, Validation Loss: 2.0219
Epoch 4, Validation Loss: 2.0011
Epoch 5, Validation Loss: 1.9783
Epoch 6, Validation Loss: 1.9540
Epoch 7, Validation Loss: 1.9259
Epoch 8, Validation Loss: 1.8951
Epoch 9, Validation Loss: 1.8606
Epoch 10, Validation Loss: 1.8221
Validation Accuracy: 0.5787


# Task ID77: 146817

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=146817)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.8724
Epoch 2, Validation Loss: 1.8041
Epoch 3, Validation Loss: 1.7286
Epoch 4, Validation Loss: 1.6425
Epoch 5, Validation Loss: 1.5520
Epoch 6, Validation Loss: 1.4630
Epoch 7, Validation Loss: 1.3808
Epoch 8, Validation Loss: 1.3095
Epoch 9, Validation Loss: 1.2487
Epoch 10, Validation Loss: 1.1969
Validation Accuracy: 0.5424


# Task ID78: 146819

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=146819)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6470
Epoch 2, Validation Loss: 0.6295
Epoch 3, Validation Loss: 0.6127
Epoch 4, Validation Loss: 0.5961
Epoch 5, Validation Loss: 0.5802
Epoch 6, Validation Loss: 0.5644
Epoch 7, Validation Loss: 0.5490
Epoch 8, Validation Loss: 0.5338
Epoch 9, Validation Loss: 0.5189
Epoch 10, Validation Loss: 0.5042
Validation Accuracy: 0.9074


# Task ID79: 146821

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=146821)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.3637
Epoch 2, Validation Loss: 1.3112
Epoch 3, Validation Loss: 1.2607
Epoch 4, Validation Loss: 1.2085
Epoch 5, Validation Loss: 1.1542
Epoch 6, Validation Loss: 1.0967
Epoch 7, Validation Loss: 1.0378
Epoch 8, Validation Loss: 0.9819
Epoch 9, Validation Loss: 0.9317
Epoch 10, Validation Loss: 0.8885
Validation Accuracy: 0.6792


# Task ID80: 14954

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=14954)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.6938
Epoch 2, Validation Loss: 0.6906
Epoch 3, Validation Loss: 0.6875
Epoch 4, Validation Loss: 0.6842
Epoch 5, Validation Loss: 0.6810
Epoch 6, Validation Loss: 0.6775
Epoch 7, Validation Loss: 0.6740
Epoch 8, Validation Loss: 0.6703
Epoch 9, Validation Loss: 0.6663
Epoch 10, Validation Loss: 0.6620
Validation Accuracy: 0.6204


# Task ID81: 167141

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=167141)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.5772
Epoch 2, Validation Loss: 0.4845
Epoch 3, Validation Loss: 0.4260
Epoch 4, Validation Loss: 0.3952
Epoch 5, Validation Loss: 0.3730
Epoch 6, Validation Loss: 0.3538
Epoch 7, Validation Loss: 0.3358
Epoch 8, Validation Loss: 0.3198
Epoch 9, Validation Loss: 0.3072
Epoch 10, Validation Loss: 0.2973
Validation Accuracy: 0.8610


# Task ID82: 167140

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=167140)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 1.0362
Epoch 2, Validation Loss: 0.9816
Epoch 3, Validation Loss: 0.9197
Epoch 4, Validation Loss: 0.8465
Epoch 5, Validation Loss: 0.7579
Epoch 6, Validation Loss: 0.6573
Epoch 7, Validation Loss: 0.5527
Epoch 8, Validation Loss: 0.4574
Epoch 9, Validation Loss: 0.3820
Epoch 10, Validation Loss: 0.3258
Validation Accuracy: 0.9060


# Task ID83: 167125

In [ ]:
# Load and preprocess data
X, y = load_openml_data(task_id=167125)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Update args based on data
args["num_features"] = X_train.shape[1]
args["num_classes"] = len(np.unique(y))

# Initialize and train the model
mlp_model = initialize_mlp(params["n_layers"], args["num_features"], params["hidden_dim"], args["num_classes"], args["objective"])
trained_model = train_mlp(mlp_model, X_train, y_train, X_val, y_val, args, params)

# Make predictions and calculate accuracy
predictions = predict_mlp(trained_model, X_val, args)
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1, Validation Loss: 0.5308
Epoch 2, Validation Loss: 0.3286
Epoch 3, Validation Loss: 0.1991
Epoch 4, Validation Loss: 0.1481
Epoch 5, Validation Loss: 0.1243
Epoch 6, Validation Loss: 0.1097
Epoch 7, Validation Loss: 0.0990
Epoch 8, Validation Loss: 0.0946
Epoch 9, Validation Loss: 0.0911
Epoch 10, Validation Loss: 0.0889
Validation Accuracy: 0.9710


# Task ID84: 167124

# Task ID85: 167121

In [ ]:
pip freeze > requirements.txt